In [ ]:
import pandas as pd
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path

In [ ]:
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--disable-gpu")
# options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-cache") 
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")

In [ ]:
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
import json
import pandas as pd

json_path = "../../data/raw/article_links.json"

# Membaca file JSON
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Konversi JSON ke DataFrame
df = pd.DataFrame(data["URL"], columns=["URL"])

print(df.head())  # Cek apakah data terbaca dengan benar

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# List untuk menyimpan hasil scraping
all_titles = []
all_years = []
all_authors = []

# Batasan jumlah halaman yang ingin di-scrape (None jika ingin sampai akhir)
MAX_PAGES = 5  # Ubah sesuai kebutuhan, atau None untuk tanpa batas

for index, row in df.iterrows():
    try:
        base_url = row["URL"]  # Pastikan kolom URL ada di dataset
        print(f"\n🔍 Scraping artikel dari: {base_url}")

        page_number = 1
        while True:
            # Jika MAX_PAGES ditentukan dan batas tercapai, berhenti
            if MAX_PAGES and page_number > MAX_PAGES:
                print(f"⚠️ Mencapai batas maksimum {MAX_PAGES} halaman. Lanjut ke URL berikutnya.")
                break

            # Bangun URL untuk halaman tertentu
            url = f"{base_url}&sortType=vol-only-newest&pageNumber={page_number}"
            print(f"📄 Membuka halaman {page_number}: {url}")

            driver.get(url)

            # Tunggu elemen muncul untuk memastikan halaman telah dimuat
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "col"))
                )
            except:
                print(f"⚠️ Halaman {page_number} gagal dimuat, lanjut ke URL berikutnya.")
                break

            time.sleep(3)  # Beri jeda agar halaman termuat sempurna

            # Parse halaman dengan BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")
            articles = soup.find_all("div", class_="col result-item-align px-3")

            # Mengecek apakah artikel ditemukan
            if not articles:
                print(f"⚠️ Halaman {page_number} kosong atau tidak ditemukan artikel baru. Berhenti.")
                break  # Jika halaman kosong, hentikan scraping URL ini

            for article in articles:
                # **Scraping Judul**
                h2_tag = article.find("h2")
                title_tag = h2_tag.find("a") if h2_tag else None
                title = title_tag.text.strip() if title_tag else "Judul Tidak Ditemukan"
                all_titles.append(title)

                # **Scraping Tahun**
                year_container = article.find("div", class_="description text-base-md-lh")
                year_tag = year_container.find("span") if year_container else None
                year = year_tag.text.strip() if year_tag else "Tahun Tidak Ditemukan"
                all_years.append(year)

                # **Scraping Author**
                author_spans = article.find_all("span", attrs={"_ngcontent-ng-c893371016": True})
                authors = ", ".join(set(span.text.strip() for span in author_spans))  # Hapus duplikasi
                authors = authors.replace(";", "").strip(", ")  # Bersihkan format
                all_authors.append(authors)

            print(f"✅ Berhasil scraping {len(articles)} artikel dari halaman {page_number}")

            page_number += 1  # Lanjut ke halaman berikutnya

    except KeyError as e:
        print(f"❌ Error: Kolom 'URL' tidak ditemukan. Pastikan nama kolom sesuai. Error: {e}")
    except Exception as e:
        print(f"❌ Terjadi kesalahan saat scraping {url}: {e}")

driver.quit()

In [ ]:
# List untuk menyimpan hasil scraping
all_titles = []
all_years = []
all_authors = []

for index, row in df.iterrows():
    try:
        base_url = row["URL"]  # Pastikan kolom URL ada di dataset
        print(f"\n🔍 Scraping artikel dari: {base_url}")

        page_number = 1
        while True:
            # Bangun URL untuk halaman tertentu
            url = f"{base_url}&sortType=vol-only-newest&pageNumber={page_number}"
            print(f"📄 Membuka halaman {page_number}: {url}")

            driver.get(url)

            # Tunggu elemen muncul untuk memastikan halaman telah dimuat
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "col"))
                )
            except:
                print(f"⚠️ Halaman {page_number} gagal dimuat, lanjut ke URL berikutnya.")
                break

            time.sleep(3)  # Beri jeda agar halaman termuat sempurna

            # Parse halaman dengan BeautifulSoup
            soup = BeautifulSoup(driver.page_source, "html.parser")
            articles = soup.find_all("div", class_="col result-item-align px-3")

            # Mengecek apakah artikel ditemukan
            if not articles or len(articles) == 0:
                print(f"⚠️ Halaman {page_number} kosong atau tidak ditemukan artikel baru. Coba cek secara manual.")
                continue  # Lanjutkan ke halaman berikutnya alih-alih berhenti langsung


            for article in articles:
                # **Scraping Judul**
                h2_tag = article.find("h2")
                title_tag = h2_tag.find("a") if h2_tag else None
                title = title_tag.text.strip() if title_tag else "Judul Tidak Ditemukan"
                all_titles.append(title)

                # **Scraping Tahun**
                year_container = article.find("div", class_="description text-base-md-lh")
                year_tag = year_container.find("span") if year_container else None
                year = year_tag.text.strip() if year_tag else "Tahun Tidak Ditemukan"
                all_years.append(year)

                # **Scraping Author**
                author_spans = article.find_all("span", attrs={"_ngcontent-ng-c893371016": True})
                authors = ", ".join(set(span.text.strip() for span in author_spans))  # Hapus duplikasi
                authors = authors.replace(";", "").strip(", ")  # Bersihkan format
                all_authors.append(authors)

            print(f"✅ Berhasil scraping {len(articles)} artikel dari halaman {page_number}")

            page_number += 1  # Lanjut ke halaman berikutnya

    except KeyError as e:
        print(f"❌ Error: Kolom 'URL' tidak ditemukan. Pastikan nama kolom sesuai. Error: {e}")
    except Exception as e:
        print(f"❌ Terjadi kesalahan saat scraping {url}: {e}")

driver.quit()

In [ ]:
import json
from pathlib import Path

# Membuat DataFrame dengan tambahan kolom author
output_df = pd.DataFrame({
    'Judul': all_titles,
    'Tahun': all_years,
    'Author': all_authors
})

# Menentukan path untuk menyimpan JSON
output_json_path = "../../data/raw/scraped_articles.json"

# Membuat direktori jika belum ada
output_dir = Path(output_json_path).parent
output_dir.mkdir(parents=True, exist_ok=True)

# Menyimpan DataFrame dalam format JSON (dengan indentasi untuk keterbacaan)
output_df.to_json(output_json_path, orient="records", indent=4)

print(f"\n✅ Hasil scraping telah disimpan ke: {output_json_path}")

In [ ]:
import json
from pathlib import Path

# Membuat DataFrame dengan tambahan kolom author
output_df = pd.DataFrame({
    'Judul': all_titles,
    'Tahun': all_years,
    'Author': all_authors
})

# Menentukan path untuk menyimpan JSON
output_json_path = "../../data/raw/scraped_articles.json"

# Membuat direktori jika belum ada
output_dir = Path(output_json_path).parent
output_dir.mkdir(parents=True, exist_ok=True)

# Menyimpan DataFrame dalam format JSON (dengan indentasi untuk keterbacaan)
output_df.to_json(output_json_path, orient="records", indent=4)

print(f"\n✅ Hasil scraping telah disimpan ke: {output_json_path}")